# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. 

The `torch.nn` namespace provides all the building blocks to build own neural network. 

Every module in PyTorch subclasses the nn.Module. 

A neural network is a module itself that consists of other modules (layers). 

This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get the Device

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## Define the Class

Define our neural network by subclassing `nn.Module`, and initialize the neural network layers in __init__. Every nn.

Module subclass implements the operations on input data in the `forward` method.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.



In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


## Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce `nonlinearity`, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0801,  0.4212, -0.7831, -0.1903, -0.3560,  0.3721, -0.0592, -0.2083,
         -0.2691, -0.3540,  0.3067, -0.0600, -0.2796, -0.2042,  0.3594, -0.1540,
          0.1423, -0.1660,  0.2277, -0.2779],
        [ 0.1738,  0.2016, -0.2750, -0.4872, -0.3876,  0.2636, -0.2491, -0.1575,
         -0.4983, -0.1514,  0.1351,  0.3324,  0.0271, -0.1051,  0.1627, -0.0110,
         -0.1844, -0.2109,  0.1644, -0.2473],
        [ 0.0711,  0.0664, -0.7748, -0.3509, -0.1646,  0.8067, -0.3009, -0.0146,
         -0.0539,  0.2875,  0.4019,  0.1736, -0.0924, -0.3377,  0.3676,  0.0241,
          0.0385, -0.0333,  0.2796, -0.3291]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.4212, 0.0000, 0.0000, 0.0000, 0.3721, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3067, 0.0000, 0.0000, 0.0000, 0.3594, 0.0000, 0.1423, 0.0000,
         0.2277, 0.0000],
        [0.1738, 0.2016, 0.0000, 0.0000, 0.0000, 0.2636, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1351, 0.3324, 0.0271, 0.0000, 0.16

### `nn.Sequential`

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.



In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### `nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0324,  0.0222, -0.0136,  ..., -0.0044, -0.0004, -0.0082],
        [-0.0065, -0.0281,  0.0010,  ...,  0.0017, -0.0049,  0.0325]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0140, -0.0274], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0265,  0.0355, -0.0083,  ...,  0.0303, -0.0434,  0.0153],
        [ 0.0304,  0.0005,  0.0386,  ...,  0.0176,  0.0290,  0.0272]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 